In [96]:
%reset -sf

In [97]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# available on Google, AtCoder Python3, not available on Codeforces
# import numpy as np
# import scipy

M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
# d8 = [(1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)]
# d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [124]:
# trr = ""
srr = """
38006F45291200
""".strip()


# trr = ""
with open("/Users/hkmac/Downloads/input.txt") as f:
    srr_inp = f.read().strip()
    
# arr = [[int(x) for x in row] for row in srr.split("\n")]

In [274]:
HEX_TO_BINARY_CONVERSION_TABLE = {
                              '0': '0000',

                              '1': '0001',

                              '2': '0010',

                              '3': '0011',

                              '4': '0100',

                              '5': '0101',

                              '6': '0110',

                              '7': '0111',

                              '8': '1000',

                              '9': '1001',

                              'a': '1010',

                              'b': '1011',

                              'c': '1100',

                              'd': '1101',

                              'e': '1110',

                              'f': '1111'}

def hex_to_binary(hex_string):
    binary_string = ""
    for character in hex_string:
        binary_string += HEX_TO_BINARY_CONVERSION_TABLE[character]
    return binary_string

In [282]:
from functools import reduce
from operator import *


with open("/Users/hkmac/Downloads/input.txt", 'r') as f:
    data = f.read().strip()

bits = [(int(c, 16) >> (3 - i)) & 1 for c in data for i in range(4)][::-1]

as_num = lambda x: sum(2 ** i * b for i, b in enumerate(x[::-1]))

def read_bits(data, n):
    for _ in range(n):
        yield data.pop()

def read_num(data, n):
    return as_num(list(read_bits(data, n)))

vnum_total = 0

def decode(data):
    global vnum_total
    version = read_num(data, 3)
    vnum_total += version
    type_id = read_num(data, 3)
    f = [add, mul, min, max, lambda x, y: 16 * x + y, gt, lt, eq][type_id]
    def get_subpackets():
        if type_id == 4:
            while True:
                done = not data.pop()
                yield read_num(data, 4)
                if done:
                    return
        lid = data.pop()
        if lid:
            for _ in range(read_num(data, 11)):
                yield decode(data)
        else:
            blen = read_num(data, 15)
            l1 = len(data) - blen
            while len(data) != l1:
                yield decode(data)
    return reduce(f, get_subpackets())

print(decode(bits))
print(vnum_total)

186189840660
854


In [278]:
solve("D2FE28")



solving 110100101111111000101000
remaining srr 110100101111111000101000
6 4 000


6

In [279]:
solve("38006F45291200")



solving 00111000000000000110111101000101001010010001001000000000
remaining srr 00111000000000000110111101000101001010010001001000000000

solving 110100010100101001000100100
remaining srr 110100010100101001000100100
6 4 0101001000100100
remaining srr 0101001000100100
2 4 
1 6 0 27 


9

In [281]:
solve("EE00D40C823060")



solving 11101110000000001101010000001100100000100011000001100000
remaining srr 11101110000000001101010000001100100000100011000001100000

solving 010
remaining srr 010


ValueError: invalid literal for int() with base 2: ''

In [272]:
solve("8A004A801A8002F478")



solving 100010100000000001001010100000000001101010000000000000101111010001111000
remaining srr 100010100000000001001010100000000001101010000000000000101111010001111000

solving 0


UnboundLocalError: local variable 'res' referenced before assignment

In [185]:
solve("C0015000016115A2E0802F182340")


srr 1100000000000001010100000000000000000001011000010001010110100010111000001000000000101111000110000010001101
srr 000000000000000001011000010001010110100010111000001000000000101111000110000010001101
srr 00010001010110100010111000001000000000101111000110000010001101
0 4 110100010111000001000000000101111000110000010001101
110100010111000001000000000101111000110000010001101
0
srr 110100010111000001000000000101111000110000010001101
6 4 1000001000000000101111000110000010001101
1000001000000000101111000110000010001101
6
srr 1000001000000000101111000110000010001101
srr 1111000110000010001101
7 4 00010001101
00010001101
7
srr 00010001101
0 4 

7
srr 00010001101
0 4 

0
4 0 1 2 

17
srr 00010111000001000000000101111000110000010001101
srr 00000101111000110000010001101
srr 0001101


ValueError: invalid literal for int() with base 2: ''

In [ ]:
solve("A0016C880162017C3686B18A3D4780")
